# Simple inference example with CroCo-Stereo or CroCo-Flow

In [ ]:
# Copyright (C) 2022-present Naver Corporation. All rights reserved.
# Licensed under CC BY-NC-SA 4.0 (non-commercial use only).

First download the model(s) of your choice by running
```
bash stereoflow/download_model.sh crocostereo.pth
bash stereoflow/download_model.sh crocoflow.pth
```

In [ ]:
import torch
use_gpu = torch.cuda.is_available() and torch.cuda.device_count()>0
device = torch.device('cuda:0' if use_gpu else 'cpu')
import matplotlib.pylab as plt

In [ ]:
from stereoflow.test import _load_model_and_criterion
from stereoflow.engine import tiled_pred
from stereoflow.datasets_stereo import img_to_tensor, vis_disparity
from stereoflow.datasets_flow import flowToColor
tile_overlap=0.7 # recommended value, higher value can be slightly better but slower

### CroCo-Stereo example

In [ ]:
image1 = np.asarray(Image.open('<path_to_left_image>'))
image2 = np.asarray(Image.open('<path_to_right_image>'))

In [ ]:
model, _, cropsize, with_conf, task, tile_conf_mode = _load_model_and_criterion('stereoflow_models/crocostereo.pth', None, device)


In [ ]:
im1 = img_to_tensor(image1).to(device).unsqueeze(0)
im2 = img_to_tensor(image2).to(device).unsqueeze(0)
with torch.inference_mode():
    pred, _, _ = tiled_pred(model, None, im1, im2, None, conf_mode=tile_conf_mode, overlap=tile_overlap, crop=cropsize, with_conf=with_conf, return_time=False)
pred = pred.squeeze(0).squeeze(0).cpu().numpy()

In [ ]:
plt.imshow(vis_disparity(pred))
plt.axis('off')

### CroCo-Flow example

In [ ]:
image1 = np.asarray(Image.open('<path_to_first_image>'))
image2 = np.asarray(Image.open('<path_to_second_image>'))

In [ ]:
model, _, cropsize, with_conf, task, tile_conf_mode = _load_model_and_criterion('stereoflow_models/crocoflow.pth', None, device)


In [ ]:
im1 = img_to_tensor(image1).to(device).unsqueeze(0)
im2 = img_to_tensor(image2).to(device).unsqueeze(0)
with torch.inference_mode():
    pred, _, _ = tiled_pred(model, None, im1, im2, None, conf_mode=tile_conf_mode, overlap=tile_overlap, crop=cropsize, with_conf=with_conf, return_time=False)
pred = pred.squeeze(0).permute(1,2,0).cpu().numpy()

In [ ]:
plt.imshow(flowToColor(pred))
plt.axis('off')